# Enumerate model solution space

This notebook loads the levers data and model, and tries to enumerate all the solutions in the solution space (i.e. all combinations of lever settings).

In [ ]:
import numpy as np
import logging
import re
import pandas as pd
import json

In [ ]:
%load_ext autoreload
%autoreload 2

## Define levers

Load from `levels.xlsx`

In [ ]:
from load_levers import read_levers
levers = read_levers("levers.xlsx")

## Model

In [ ]:
import load_model
model_data = load_model.load_model()

In [ ]:
model, recipe_data = load_model.build_model(model_data)
other_results = load_model.define_model(model, recipe_data, levers)

In [ ]:
flows_sym = model.to_flows(recipe_data, flow_ids=True)
func = model.lambdify(recipe_data)
func_other = model.lambdify(data=recipe_data, expressions=other_results)

# Use lever levels to get test parameter settings

If we want to see what the model output looks like for specific lever settings, use them here to find the parameter settings that can go into the model.

This duplicates the logic that is built into the interactive calculator app, which does this to re-run the model every time the levers are changed.

In [ ]:
def get_model_output(lever_settings, time_index):
    params = levers.get_params(lever_settings, time_index=time_index)
    result = func_other(params)
    return result

In [ ]:
baseline = {lever.lever_id: lever.levels[0].level_id for lever in levers.levers}
get_model_output(baseline, 0)

## Enumerate all lever settings

In [ ]:
def enumerate_lever_settings(levers):
    if not levers:
        yield {}
        return
    next_lever = levers[0]
    rest_levers = levers[1:]
    for level in next_lever.levels:
        for settings in enumerate_lever_settings(rest_levers):
            yield {**settings, next_lever.lever_id: level.level_id}
        

In [ ]:
len([lever for lever in levers.levers if len(lever.levels) > 1])

How many combinations do we expect?

In [ ]:
n_combinations = np.product([min(5, len(lever.levels)) for lever in levers.levers])
print(f"{n_combinations/1e6:.0f} million combinations")

In [ ]:
test_params = levers.get_params(baseline, time_index=6)

In [ ]:
%%timeit
func_other(test_params)

In [ ]:
%%timeit
get_model_output(baseline, 6)

Approx 4832 million combinations, approx 6 ms/results, makes:

In [ ]:
print(f"{4830e6 * 6e-3 / 60 / 60 / 24} days")

In [ ]:
print(f"{1e6 * 6e-3 / 60 / 60} hours")

In [ ]:
# results = pd.DataFrame([
#     {**settings, **get_model_output({**baseline, **settings}, time_index=6)}
#     for settings in enumerate_lever_settings(levers.levers[:])
# ])

In [ ]:
#results = pd.read_csv("enumerated_results_6.csv", index_col=list(range(len(baseline))))
results = pd.read_parquet(f"../outputs/enumerated_results_6.parquet")#, columns=["GHG_total"])

In [ ]:
results.iloc[0]

Simple SA:
- Morris style effect of the change from level 1 to level 4 for each lever: lots of results for all the other permutations.
- Summarise in a histogram what the effect is
    - Some are always positive
    - Some, it depends, etc
    
    
?

In [ ]:
def effect_calc(lever):
    levels = results.index.levels[results.index.names.index(lever)]
    a = results["GHG_total"].xs(levels[0], level=lever)
    b = results["GHG_total"].xs(levels[-1], level=lever)
    effect = (b - a) / 1e12  # Gt
    return {
        "whislo": effect.min(),
        "q1":  np.quantile(effect, 0.25),
        "med": np.quantile(effect, 0.50),
        "q3":  np.quantile(effect, 0.75),
        "whishi": effect.max(),
        "mu": effect.mean(),
        "mu_star": abs(effect).mean(),
        "sigma": effect.std(),
        "label": lever,
    }

In [ ]:
lever_ids = [lever.lever_id for lever in levers.levers]
ee = pd.DataFrame({
    lever_id: effect_calc(lever_id)
    for lever_id in lever_ids
    # Don't include levers that only have one level
    if len(results.index.levels[results.index.names.index(lever_id)]) > 1
}).T

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!mkdir -p figures

In [ ]:
fig, ax = plt.subplots()
ee_sorted = ee.reset_index().sort_values("mu_star", ascending=True)
ax.bxp(ee_sorted.to_dict('records'), showfliers=False, vert=False);
ax.axvline(0, alpha=0.3);
ax.set_xlabel("Change in emissions caused by varying each lever [GtCO2e]");
ax.set_ylabel("Lever ID");
fig.savefig("figures/lever_sensitivity.png", dpi=200);
fig.savefig("figures/lever_sensitivity.pdf");

In [ ]:
# NB currently only have min/max levels
shape = [min(2, len(lever.levels)) for lever in levers.levers]

In [ ]:
(results["GHG_total"] / 1e9).hist(bins=40)

In [ ]:
GHG_total = results.reset_index()
GHG_total["class"] = "High"
GHG_total.loc[GHG_total["GHG_total"] < 0.3e12, "class"] = "Low"

In [ ]:
threshold = 100

cols_low = GHG_total[GHG_total["GHG_total"] < (threshold * 1e9)].iloc[:, :len(levers.levers)]
cols_high = GHG_total[GHG_total["GHG_total"] >= (threshold * 1e9)].iloc[:, :len(levers.levers)]

for col in cols_low.columns:
    levels_low = cols_low[col].unique()
    levels_high = cols_high[col].unique()
    if len(levels_low) < len(levels_high):
        print(col)
        print("Low emissions:   ", levels_low)
        print("All combinations:", levels_high)
        print()

In [ ]:
results["GHG_total"].min() / 1e12

In [ ]:
results["GHG_total"].max() / 1e12

In [ ]:
print("\n".join(results.columns))

Variation in emissions factor with production quantity? Is it linear?

This is not an exact calculation of the emissions factor, which would require LCA calculation on top of the supply mixes from the mass flow model. But approximately.

In [ ]:
results["Emissions_polymer_production"] = (
    results["EmissionsByStage_feedstocks"] +
    results["EmissionsByStage_hydrogen"] +
    results["EmissionsByStage_primary_production"] +
    results["EmissionsByStage_organic_synthesis"] +
    results["EmissionsByStage_downstream"] +
    results["EmissionsByStage_biomass"]
)
# Excluding biomass (-1) and end-of-life (+1)

In [ ]:
results["Production_polymers_virgin_label"] = results["Production_polymers_virgin"].apply(lambda x: x/1e6) #f"{x/1e6:.0f} Mt")
results["Emissions_polymer_production_per_kg"] = results["Emissions_polymer_production"] / (1000 * results["Production_polymers_virgin"])

In [ ]:
results_fewer = results.xs("1", level="extra_demand")

In [ ]:
sns.set_style("darkgrid");

In [ ]:
#plt.boxplot(

In [ ]:
plt.axhline(0, color='k', lw=0.5, alpha=0.6);
plt.axhline(2.67, color='C1', lw=1);
plt.text(300, 2.8, "Plastsimulator", color='C1');
sns.boxplot(results_fewer, x="Production_polymers_virgin_label", y="Emissions_polymer_production_per_kg", native_scale=True, flierprops=dict(alpha=0.006));
plt.xlabel("Virgin polymer production [Mt]");
plt.ylabel("Production emissions factor [kgCO2e/kg]");

In [ ]:
threshold = 15.5

GHG_total = results_fewer.reset_index()

cols_low = GHG_total[GHG_total["Emissions_polymer_production_per_kg"] < (threshold)].iloc[:, :len(levers.levers)]
cols_high = GHG_total[GHG_total["Emissions_polymer_production_per_kg"] >= (threshold)].iloc[:, :len(levers.levers)]

for col in cols_low.columns:
    levels_low = cols_low[col].unique()
    levels_high = cols_high[col].unique()
    if len(levels_high) < len(levels_low):
        print(col)
        print("High emissions:   ", levels_high)
        print("All combinations:", levels_low)
        print()

In [ ]:
cols_high

In [ ]:
sns.scatterplot(results, x="EmissionsBySource_Elec", y="EmissionsBySource_NG", hue="ethylene_methanol_capacity", alpha=0.5);

In [ ]:
sns.scatterplot(results, x="ElecReq", y="NGReq", hue="ethylene_methanol_capacity", alpha=0.5);

In [ ]:
sns.scatterplot(results, x="ElecReq", y="NGReq", hue="product_demand", alpha=0.5);

In [ ]:
results["ambition"] = np.log(results[baseline.keys()].astype(int).product(axis=1))

In [ ]:
sns.scatterplot(results, x="ElecReq", y="NGReq", hue="ambition", alpha=0.5);

In [ ]:
years = list(range(2020, 2051, 5))
over_time = pd.DataFrame([
    pd.read_parquet(f"enumerated_results_{i}.parquet", columns=["GHG_total"])["GHG_total"]
    for i in range(7)
], index=years).T

In [ ]:
d = (over_time / 1e9).values.T
d.shape

In [ ]:
plt.plot(years, d[:, ::51], alpha=0.005, c='k');

In [ ]:
sns.set_theme(style='darkgrid')

In [ ]:
def plot_scenarios_over_time(ax):
    ylim = (-0.2, 6.2)

    ax.plot(np.arange(7)/7, d[:, ::51]/1e3, alpha=0.005, c='k');
    ax.plot(np.arange(7)/7, d[:, 0]/1e3, lw=2, c='r');
    ax.set_xlim(0, 1);
    ax.set_ylim(ylim);
    
    bins = np.arange(0, 5, 0.1);
    for i in range(7):
        axins = ax.inset_axes([i/7, 0, 1/10, 1], ylim=ylim, xticks=[], yticklabels=[])
        axins.patch.set_alpha(0.5)
        axins.grid(False)
        axins.set_frame_on(False)
        axins.hist(d[i]/1000, bins=bins, orientation='horizontal', density=False)
        
        # Show the range, because otherwise it doesn't look like the baseline part is included
        axins.plot([0, 0], [d[i].min()/1000, d[i].max()/1000], "C0", marker='_')
        
        #ax[i].set_frame_on(False)
        #ax[i].xaxis.set_visible(False)
        #ax[i].set_xticks([])
        #ax[i].set_xlabel(years[i])
        #if i > 0:
        #    ax[i].yaxis.set_ticks_position("none")
        axins.plot(0, d[i, 0]/1000, 'ro', ms=5, clip_on=False);
    ax.set_xticks([i/7 for i in range(7)])
    ax.set_xticklabels([2020+i*5 for i in range(7)])
#    ax.axhline(0, c='lightgreen', lw=0.75);
    ax.axhspan(ylim[0], 0, color="C2", alpha=0.2);
    ax.set_xlabel("Year");
    ax.set_ylabel("Total GHG emissions [GtCO2e]");
    axins.annotate("Baseline", xy=(0, d[-1, 0]/1000), xytext=(8, 0), textcoords='offset points', c='r', va='center');

fig, ax = plt.subplots() #7, sharey=True, sharex=False)
plot_scenarios_over_time(ax)    
fig.savefig("figures/scenarios_over_time.png", dpi=200);
fig.savefig("figures/scenarios_over_time.pdf");

In [ ]:
contribution_columns = [
    "EmissionsByStage_end_of_life",
    "GHG_use_fertiliser",
    "EmissionsByStage_downstream",
    "EmissionsByStage_organic_synthesis",
    "EmissionsByStage_primary_production",
    "GHG_production_fertiliser",
    "EmissionsByStage_hydrogen",
    "EmissionsByStage_biomass",
    "EmissionsByStage_feedstocks",
]
contributions_over_time_baseline = pd.DataFrame([
    pd.read_parquet(f"enumerated_results_{i}.parquet", columns=contribution_columns).iloc[0]
    for i in range(7)
], index=years).T

In [ ]:
contributions_over_time_baseline

In [ ]:
def plot_baseline_contributions(ax):
    results = contributions_over_time_baseline.T
    df = pd.DataFrame({
        "Fertiliser production": results["GHG_production_fertiliser"],
        "Fertiliser use": results["GHG_use_fertiliser"],
        "Feedstock": results["EmissionsByStage_feedstocks"],
        "Biomass": results["EmissionsByStage_biomass"],
        "Hydrogen production": results["EmissionsByStage_hydrogen"],
        "Primary chemicals": results["EmissionsByStage_primary_production"],
        "Downstream chemicals": results["EmissionsByStage_downstream"] + results["EmissionsByStage_organic_synthesis"],
        "End-of-life": results["EmissionsByStage_end_of_life"],
    }) / 1e12

    total = df.sum(axis=1)
    
    palette = ["#4269D0","#97BBF5",  "#9C6B4E", "#9498A0", "#EFB118", "#FF725C","#FF8AB7",
              "#6CC5B0", "#3CA951", 
              "#A463F2", 
              ]
    df.plot.area(stacked=True, color=palette, legend=False, alpha=0.8, ax=ax)
    ax.plot(df.index, total, lw=2, c="k", label="Total")
    #ax.legend(bbox_to_anchor=(1.0, 1.0))
    ax.set_xlim(2020, 2050)
    ax.set_xlabel("Year")
    ax.set_ylabel("Total GHG emissions [GtCO2e]")
    ax.axhline(0, lw=0.5, c='k')
    
    xx = df.loc[2050]
    xx_pos = xx[xx > 0]
    y_top = np.cumsum(xx_pos)
    y_centre = y_top - np.diff(y_top, prepend=0) / 2
    labels = xx_pos.index
    palette_dict = dict(zip(df.columns, palette))
    palette_pos = [palette_dict[label] for label in labels]
    for k, v, c in zip(labels, y_centre, palette_pos):
        if abs(v) > 0.002:
            ax.annotate(k, xy=(2050, v), xytext=(8, 0), clip_on=False,
                        fontsize=10,
                        textcoords='offset points', c=c, va='center');

fig, ax = plt.subplots()
plot_baseline_contributions(ax)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 4), sharey=True)
plot_scenarios_over_time(ax[0])    
plot_baseline_contributions(ax[1])
ax[0].set_title("(a) Range of scenarios")
ax[1].set_title("(b) Contributions to baseline scenario")
fig.savefig("figures/scenarios.png", dpi=200);
fig.savefig("figures/scenarios.pdf");